In [3]:
import heapq
h = [5, 2, 8]
heapq.heapify(h)  # Make 'h' a heap

smallest = heapq.heappop(h)  # removes 2 (the smallest)
print(smallest)  # Output: 2
h.append(4)  # Add 4 to the heap
heapq.heapify(h)  # Re-heapify after appending
print(h)  # Output: [4, 5, 8] (the heap structure is maintained)

2
[4, 8, 5]


In [1]:
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from geometric_sampling.design import Design
import numpy as np

# 8 inclusions summing to 3 (example)
inclusions = np.array([0.5, 0.5, 0.3, 0.2, 0.6, 0.9])  # sum = 3.0
print("Sum of inclusions:", inclusions.sum())

# (A) No perm, no atomization
d_classic = Design(inclusions=inclusions)
print("\n== (A) Classic (no perm, no atom) ==")
for s in d_classic:
    print(s)

# (B) With permutation, no atomization
perm = np.random.permutation(len(inclusions))
d_perm = Design(inclusions=inclusions, perm=perm)
print("\n== (B) Permutation only (no atom) ==")
print("Permutation:", perm)
for s in d_perm:
    print(s)

print(inclusions[5])


Sum of inclusions: 3.0

== (A) Classic (no perm, no atom) ==
Sample(probability=np.float64(0.5), ids=frozenset({1, 4, 5}))
Sample(probability=np.float64(0.2), ids=frozenset({0, 2, 5}))
Sample(probability=np.float64(0.2), ids=frozenset({0, 3, 5}))
Sample(probability=np.float64(0.1), ids=frozenset({0, 2, 4}))

== (B) Permutation only (no atom) ==
Permutation: [4 1 2 0 3 5]
Sample(probability=np.float64(0.3), ids=frozenset({2, 4, 5}))
Sample(probability=np.float64(0.3), ids=frozenset({0, 1, 5}))
Sample(probability=np.float64(0.2), ids=frozenset({0, 4, 5}))
Sample(probability=np.float64(0.1), ids=frozenset({1, 3, 4}))
Sample(probability=np.float64(0.1), ids=frozenset({1, 3, 5}))
0.9


In [20]:
import numpy as np

class WindowedAtomizedDesign:
    def __init__(self, inclusions, k, window_size=None, rng=None, delta=1e-2):
        """
        inclusions: list of floats (the original unit weights)
        k: int (design size = window size usually)
        window_size: int or None, if None, use k
        rng: np.random.Generator instance
        delta: atom granularity
        """
        self.rng = rng if rng is not None else np.random.default_rng()
        self.delta = delta
        self.inclusions = np.array(inclusions)
        self.n_units = len(self.inclusions)
        self.window_size = window_size or k
        self.k = k
        # Step 1: Permute units and keep map
        self.perm = self.rng.permutation(self.n_units)
        self.unit_map = {new: orig for new, orig in enumerate(self.perm)}
        self.perm_inclusions = self.inclusions[self.perm]
        # Step 2: Atomize each permuted inclusion
        self.atoms = []
        self.atom_to_unit = []  # which permuted unit does each atom come from
        for i, p in enumerate(self.perm_inclusions):
            n_full = int(p // delta)
            for _ in range(n_full):
                self.atoms.append(delta)
                self.atom_to_unit.append(i)
            remainder = p - n_full * delta
            if remainder > 1e-12:
                self.atoms.append(remainder)
                self.atom_to_unit.append(i)
        self.atoms = np.array(self.atoms)
        self.atom_to_unit = np.array(self.atom_to_unit)
        self.n_atoms = len(self.atoms)

    def sample_design(self):
        """
        For a single design:
        - Split atoms into windows of given 'window_size'.
        - In each window, randomly permute atom order (leaving border outside window, if total atoms not divisible).
        - Select first atom from each window (fixed design size = number of windows).
        - Return: [probability, set of original_unit_indices] (map through permutations)
        """
        n_windows = self.n_atoms // self.window_size
        selected_atom_indices = []
        for w in range(n_windows):
            start = w * self.window_size
            end = (w+1) * self.window_size
            window_indices = np.arange(start, end)
            permuted_indices = self.rng.permutation(window_indices)
            # select first in permuted order (fixed one per window)
            selected_atom_indices.append(permuted_indices[0])
        # fixed design size = n_windows (could also do window sampling for k)
        sel_atom_units = [self.atom_to_unit[idx] for idx in selected_atom_indices]
        # Map back to original units
        orig_units = [self.unit_map[u] for u in sel_atom_units]
        # Collect their atom's probability weights:
        probs = [self.atoms[idx] for idx in selected_atom_indices]
        total_prob = sum(probs)
        return total_prob, frozenset(orig_units), probs

    def many_designs(self, N=10):
        return [self.sample_design() for _ in range(N)]

if __name__ == "__main__":
    inclusions = [0.4, 1.0, 0.6, 0.8, 0.1, 0.1]  # example: sum = 3.0
    k = 3  # design size, also used for window size

    design = WindowedAtomizedDesign(inclusions, k)
    print(f"\nPermutation: {design.perm}")
    print(f"Unit map (new->orig): {design.unit_map}")
    print(f"n_atoms: {design.n_atoms}, atom to unit: {design.atom_to_unit}")
    print("Example designs (prob, orig_unit_ids, atom_probs):")
    for d in design.many_designs(5):
        print(d)


Permutation: [2 0 1 3 5 4]
Unit map (new->orig): {0: np.int64(2), 1: np.int64(0), 2: np.int64(1), 3: np.int64(3), 4: np.int64(5), 5: np.int64(4)}
n_atoms: 300, atom to unit: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5
 5 5 5 5]
Example designs (prob, orig_unit_ids, atom_probs):
(np.float64(1.0000000000000007), frozenset({np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)}), [np.float64(0.01), np.float64(0.01), np.

In [1]:
import numpy as np
x = np.array([1, 2, 1, 6, 7, 3, 8])
perm = np.argsort(x)
x[perm]
np.argsort(np.array(range(10)))  # increasing order of x


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [28]:
from geometric_sampling.design import Design
import numpy as np

In [29]:
# ======
# DEMO
# ======

inclusions = np.array([.2, .3, .45, .85, .2])
print("=== Case 1: Classical (no perm, no delta)===")
d1 = Design(inclusions=inclusions, delta=1.0)
for sample in d1:
    print(sample)

print("\n=== Case 2: Random UNIT perm (perm on inclusions, NO delta)===")
perm = np.random.permutation(len(inclusions))
d2 = Design(inclusions=inclusions[perm], delta=1.0)
for sample in d2:
    print(sample)

print("\n=== Case 3: Atomized, random perm on ALL atoms, delta<1.0 ===")
d3 = Design(inclusions=inclusions, delta=0.05)
for sample in d3:
    print(sample)

=== Case 1: Classical (no perm, no delta)===
Sample(probability=np.float64(0.3), ids=frozenset({1, 3}))
Sample(probability=np.float64(0.2), ids=frozenset({0, 3}))
Sample(probability=np.float64(0.3), ids=frozenset({2, 3}))
Sample(probability=np.float64(0.15), ids=frozenset({2, 4}))
Sample(probability=np.float64(0.05), ids=frozenset({3, 4}))

=== Case 2: Random UNIT perm (perm on inclusions, NO delta)===
Sample(probability=np.float64(0.45), ids=frozenset({2, 4}))
Sample(probability=np.float64(0.3), ids=frozenset({0, 2}))
Sample(probability=np.float64(0.15), ids=frozenset({1, 3}))
Sample(probability=np.float64(0.05), ids=frozenset({2, 3}))
Sample(probability=np.float64(0.05), ids=frozenset({1, 2}))

=== Case 3: Atomized, random perm on ALL atoms, delta<1.0 ===
Sample(probability=np.float64(0.05), ids=frozenset({0, 3}))
Sample(probability=np.float64(0.05), ids=frozenset({0, 3}))
Sample(probability=np.float64(0.05), ids=frozenset({0, 3}))
Sample(probability=np.float64(0.05), ids=frozenset({

In [ ]:
Sample(probability=np.float64(0.3), ids=frozenset({1, 3}))
Sample(probability=np.float64(0.2), ids=frozenset({0, 3}))
Sample(probability=np.float64(0.3), ids=frozenset({2, 3}))
Sample(probability=np.float64(0.15), ids=frozenset({2, 4}))
Sample(probability=np.float64(0.05), ids=frozenset({3, 4}))